<a href="https://colab.research.google.com/github/illusion173/CS555UAV/blob/main/CS555Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Necessary Commands to run before import
%pip install ultralytics
!pip install -U ipywidgets

In [21]:
# Import needed forms
import os
import collections
import pandas as pd
import numpy as np
import functools
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
import warnings
import zipfile
from google.colab import drive
import ultralytics
import shutil
from ultralytics import YOLO, settings
import yaml

**GOOGLE DRIVE MOUNTING/UNZIPPING OF DATASET**

In [11]:
# DO NOT TOUCH PLEASE, loc of your zip file should match this
# the zip should be in the root of your google drive.

GOOGLE_DRIVE_DATA_DIR_PATH = "/content/drive/"
drive.mount(GOOGLE_DRIVE_DATA_DIR_PATH, force_remount=True)
%cd /content/drive/MyDrive/
# Unzip the file into /tmp using -d, shouldn't take too long <1 min
!unzip -q "uav_datasets.zip" -d /tmp


Mounted at /content/drive/
/content/drive/MyDrive
replace /tmp/dataset_xml_format/dataset_xml_format/foto00088.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

**DATA INGESTION/ORGANIZATION**

In [12]:
# KEEP NOTE THAT THE DIRECTORIES MADE IN THIS CELL ARE IN THE /tmp dir and WILL BE DELETED
YOLO_DATASET_FOLDER = "/tmp/drone_dataset_yolo/dataset_txt/" # This dir made in the above unzip cell
TRAIN_IMAGES_FOLDER = "/tmp/train/images"
TRAIN_LABELS_FOLDER = "/tmp/train/labels"
VAL_IMAGES_FOLDER = "/tmp/val/images"
VAL_LABELS_FOLDER = "/tmp/val/labels"
os.makedirs(TRAIN_IMAGES_FOLDER, exist_ok=True)
os.makedirs(TRAIN_LABELS_FOLDER, exist_ok=True)
os.makedirs(VAL_IMAGES_FOLDER, exist_ok=True)
os.makedirs(VAL_LABELS_FOLDER, exist_ok=True)

In [13]:
# Creating necessary lists for organizing the files
images = []
labels = []

# Collect image and label file paths from the dataset folder
for file_name in os.listdir(YOLO_DATASET_FOLDER):
    if file_name.endswith((".jpg", ".png")):  # Assuming images are .jpg or .png
        base_name = os.path.splitext(file_name)[0]
        label_file = base_name + ".txt"  # Assuming labels are in YOLO format (.txt)

        # Check if the corresponding label file exists
        if os.path.exists(os.path.join(YOLO_DATASET_FOLDER, label_file)):
            images.append(file_name)
            labels.append(label_file)

In [14]:
# Split the data into training and validation sets 80% train, 20% val
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)


In [15]:
# Define helper function to organize the files into appropriate train and value folders
def move_files(file_list, src_folder, dest_folder):
    for file in file_list:
        shutil.copy(os.path.join(src_folder, file), os.path.join(dest_folder, file))

In [16]:
# Move the training images and labels to the train folder
move_files(train_images, YOLO_DATASET_FOLDER, TRAIN_IMAGES_FOLDER)
move_files(train_labels, YOLO_DATASET_FOLDER, TRAIN_LABELS_FOLDER)

# Move the validation images and labels to the val folder
move_files(val_images, YOLO_DATASET_FOLDER, VAL_IMAGES_FOLDER)
move_files(val_labels, YOLO_DATASET_FOLDER, VAL_LABELS_FOLDER)

Model Creation

In [24]:
yolo_config = {
    "train": os.path.join("/tmp", "train/images"),
    "val": os.path.join("/tmp", "val/images"),
    "nc": 1,
    "names": ["drone"]
}
# THIS FILE WILL BE DELETED, REQUIRED TO BE RAN EACH TIME.
# Define the output YAML file path
yaml_file = "/tmp/yolo_dataset.yaml"

# Write the YOLO dataset information to the YAML file
with open(yaml_file, "w") as file:
    yaml.dump(yolo_config, file)

In [27]:
# Physical Creation of the yolo model
# We don't need wandb
settings.update({"wandb": False})

# Load a COCO-pretrained YOLOv8n model
yolo_model = YOLO("yolo11n.pt")

yolo_model.info()

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(181, 2624080, 0, 6.614336)

Model Training

In [30]:
results = yolo_model.train(
    data = yaml_file,
    epochs = 100, # We can default to 100 for now then on patience setting will tell it to stop after 10 epochs if it isn't getting any better
    imgsz = 640,
    batch = -1,
    optimizer = "AdamW"
    patience = 10
)
# Docs here
# https://docs.ultralytics.com/modes/train/

Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/tmp/yolo_dataset.yaml, epochs=1, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, 

train: Scanning /tmp/train/labels.cache... 1087 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1087/1087 [00:00<?, ?it/s]

train: WARNING ⚠️ /tmp/train/images/pic_722.jpg: corrupt JPEG restored and saved
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch:  ⚠️ intended for CUDA devices, using default batch-size 16
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 603.0±806.6 MB/s, size: 602.0 KB)



train: Scanning /tmp/train/labels.cache... 1087 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1087/1087 [00:00<?, ?it/s]

train: WARNING ⚠️ /tmp/train/images/pic_722.jpg: corrupt JPEG restored and saved
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1114.6±1093.8 MB/s, size: 362.2 KB)



val: Scanning /tmp/val/labels.cache... 272 images, 0 backgrounds, 0 corrupt: 100%|██████████| 272/272 [00:00<?, ?it/s]

Plotting labels to runs/detect/train3/labels.jpg... 



optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train3
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/68 [00:02<?, ?it/s]


KeyboardInterrupt: 